In [0]:
# Імпортуємо лише необхідні функції з Spark
from pyspark.sql.functions import col, count, avg, max, min, sum, when

# Читаємо таблицю raw_trips з Unity Catalog
df_raw_trips = spark.table("vmilichenko_nyc_catalog.nyc_taxi.raw_trips")

# Додаємо допоміжні булеві колонки:
# - чи поїздка yellow
# - чи green
# - чи вартість більше $30
df_raw_trips = df_raw_trips.withColumn("is_yellow", (col("taxi_type") == "yellow").cast("double")) \
       .withColumn("is_green", (col("taxi_type") == "green").cast("double")) \
       .withColumn("is_high_fare", (col("fare_amount") > 30).cast("double"))

# -----------------------------------------------
# Створення агрегованої таблиці zone_summary
# -----------------------------------------------
zone_summary = df_raw_trips.groupBy("pickup_zone").agg(
    count("*").alias("total_trips"),                        # загальна кількість поїздок
    avg("trip_distance").alias("avg_trip_distance"),        # середня дистанція
    avg("total_amount").alias("avg_total_amount"),          # середня вартість поїздки
    avg("tip_amount").alias("avg_tip_amount"),              # середні чайові
    avg("is_yellow").alias("yellow_share"),                 # частка yellow таксі
    avg("is_green").alias("green_share"),                   # частка green таксі
    max("trip_distance").alias("max_trip_distance"),        # макс дистанція
    min("tip_amount").alias("min_tip_amount"),              # мін чайові
    sum("total_amount").alias("total_trip_amount")          # сумарна виручка
)

# Запис результату у форматі Delta до Unity Catalog
zone_summary.write.format("delta").mode("overwrite").saveAsTable("vmilichenko_nyc_catalog.nyc_taxi.zone_summary")

# -----------------------------------------------
# Створення агрегованої таблиці zone_days_summary
# -----------------------------------------------
zone_days_summary = df_raw_trips.groupBy("pickup_day_of_week", "pickup_zone").agg(
    count("*").alias("total_trips_per_day"),                # кількість поїздок за день у зоні
    avg("duration_min").alias("avg_duration_per_zone"),     # середня тривалість
    avg("is_high_fare").alias("high_fare_share")            # частка дорогих поїздок (> $30)
)

# Запис результату у форматі Delta до Unity Catalog
zone_days_summary.write.format("delta").mode("overwrite").saveAsTable("vmilichenko_nyc_catalog.nyc_taxi.zone_days_summary")
